# JSON and APIs

_August 11, 2020_

Agenda today:
- Introduction to API and Remote Server Model 
- Getting data through an API:
    - Github API
    - Yelp API
- Working with JSON files

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from yelp.client import Client
import matplotlib.pyplot as plt
plt.style.use('seaborn')

## Part I. APIs and Remote Server Model
API stands for Application Programming Interface. At some point or the other, large companies would build API for their products for their clients or internal use. It allows the company's application to communicate with another application. But what _exactly_ is an API?

#### Remote server 
When we think about the world of Web, we can think of it as a collection of _servers_. And servers are nothing but huge computers that store a huge amount of data from users and are optimized to process requests. For example, when you type in www.facebook.com, your browser sends a _request_ to the Facebook server, and gets a response from the server, thus interpreting the code and displaying your homepage. 

In this case, your browser is the _client_, and Facebook’s server is an API. To put it broadly, whenever you visit a website, you are interacting with its API. However, an API isn’t the same as the remote server — rather it is the part of the server that receives __requests__ and sends __responses__.

<img src='status-code.png' width = 500>

## Part II. Getting Data Through APIs

#### Github API
Github API is an example of API that does not need _authentication_. You can send `GET` requests to the API and receive information.  

The `get()` method send a request to Github's API, and stored information in a variable called `request`. Next, let's see if it's successful. 

In [ ]:
request = requests.get('https://api.github.com')

In [ ]:
## status code


In [ ]:
# examine the body of the request
request.text

In [ ]:
# examine the content
request.content

In [ ]:
# examine the headers
request.headers['server']

In [ ]:
# we can make use of the JSON library to efficiently load the data and manipulate it 
request_json = json.loads(request.text)

In [ ]:
request_json

In [ ]:
# loop through the request_json file and examine the keys 

# how do you loop through the keys again?
for key in request_json.keys():
    print(key)

In [ ]:
# can you think of a way in which you'd put them in a dataframe?
request_df = pd.DataFrame.from_dict(request_json, orient='index')
request_df.columns = ['url']
request_df.head()

#### YELP API
Sometimes you need _authentication_ to get data from a service in additional to just sending a `GET()` request. Yelp API is a perfect example. 

You will need to go to the YELP's developer's [website](https://www.yelp.com/developers/v3/manage_app) and request for a client ID and API key, which function like a key into a house of data. 

<img src='yelp.png' width = 500>

In [ ]:
# lets try to get some data from yelp!
url = 'https://api.yelp.com/v3/businesses/search'
response = requests.get(url)

In [ ]:
# check the status code
response.status_code

# what happened here?

In [ ]:
#You have to use your API key to access the data!

MY_API_KEY = "My Key" # replace this with your API key!
client = Client(MY_API_KEY)

In [ ]:
# now we are ready to get our data 

# usually, services would limit you to a certain amount of API calls. This varies from service
# to service, so you have to watch out to it 

term = 'Axe Throwing'
location = 'Brooklyn'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(MY_API_KEY),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text)

In [ ]:
# cleaning and formatting the data
axe_throwing = response.text
axe_throwing = json.loads(axe_throwing)

In [ ]:
# cleaning and exploring the data
for key in axe_throwing.keys():
    print(key)

In [ ]:
# examine the first element of businesses
axe_throwing['businesses'][0].keys()

In [ ]:
axe_throwing_df = pd.DataFrame.from_dict(axe_throwing['businesses'])
axe_throwing_df

In [ ]:
# you can do some analysis and visualization from here on! 

plt.hist(axe_throwing_df['review_count'], color='pink', alpha = 0.8)
plt.title('Axe Throwing reviewing count in Brooklyn')
plt.xlabel('Reviews')
plt.ylabel('Count')

In [ ]:
# query the name of the axe throwing place with the highest review
axe_throwing_df.sort_values(by = 'review_count', ascending = False).name.reset_index(drop = True)[0]


In [ ]:
# can you do some other queries using sql/pandas?

#### Resources
- [Getting Data from Reddit API](https://www.storybench.org/how-to-scrape-reddit-with-python/)
- [Twitch API](https://dev.twitch.tv/docs)